In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sys,os
import random
path = os.getcwd()

In [7]:
# rps : readingsPerSample
def exportFigs(input_filename="mHealth_subject3.log",rps=50,figsize=(5,5),output_folderName="",prefex="fig", activities = range(1,12)):
    os.chdir(path)
    
    df = pd.read_csv(input_filename, header=None, delim_whitespace=True)
    df = df[df[23]!=0] #drop 0 labeled features
    features_df = df[3]
    targets_df = df[23]
    
    #normalization
    features_np = features_df.values
    features_np_normalized = (features_np-min(features_np))/(max(features_np)-min(features_np))
    features_df_normalized = pd.DataFrame(features_np_normalized)
    
    #segmentation
    targets_dummy  = np.array([])
    features_dummy  = np.array([])
    for i in range(int(features_df_normalized.shape[0]/rps)):
        if  targets_df[i*rps:i*rps+rps].mean() in activities:
            features_dummy=np.append(features_dummy,features_df_normalized[i*rps:i*rps+rps])
            targets_dummy=np.append(targets_dummy,targets_df[i*rps:i*rps+rps].mean()-1)

    features_dummy=features_dummy.reshape(-1,rps)
    
    #folders hiararchy
    if output_folderName == "":
        output_folderName=randomString(10)
        
    os.chdir(path)
    try :
        os.makedirs(output_folderName)
    except:
        pass
    path_str = os.path.join(str(path),output_folderName)
    os.chdir(path_str)
    
    tmp=1
    imageCounter=1
    for i in range(1,len(targets_dummy)) :
        if int(targets_dummy[i]+1) != tmp:
            imageCounter = 0
            tmp=int(targets_dummy[i]+1)
        imageCounter+=1
        plt.figure(figsize=figsize)
        plt.xticks([])
        plt.yticks([])
        plt.rc("figure", facecolor="white")
        for spine in plt.gca().spines.values():
            spine.set_visible(False)
        plt.plot(features_dummy[i], color='k')
        label=str(int(targets_dummy[i]+1))
        print(label+"_fig_"+str(imageCounter)+".png")
        plt.savefig(label+"_fig_"+str(imageCounter)+".png")
        plt.clf()
        plt.cla()
        plt.close()
    os.chdir(path)

In [8]:
exportFigs(output_folderName="test",figsize=(5,5),rps=50)

1_fig_2.png
1_fig_3.png
1_fig_4.png
1_fig_5.png
1_fig_6.png
1_fig_7.png
1_fig_8.png
1_fig_9.png
1_fig_10.png
1_fig_11.png
1_fig_12.png
1_fig_13.png
1_fig_14.png
1_fig_15.png
1_fig_16.png
1_fig_17.png
1_fig_18.png
1_fig_19.png
1_fig_20.png
1_fig_21.png
1_fig_22.png
1_fig_23.png
1_fig_24.png
1_fig_25.png
1_fig_26.png
1_fig_27.png
1_fig_28.png
1_fig_29.png
1_fig_30.png
1_fig_31.png
1_fig_32.png
1_fig_33.png
1_fig_34.png
1_fig_35.png
1_fig_36.png
1_fig_37.png
1_fig_38.png
1_fig_39.png
1_fig_40.png
1_fig_41.png
1_fig_42.png
1_fig_43.png
1_fig_44.png
1_fig_45.png
1_fig_46.png
1_fig_47.png
1_fig_48.png
1_fig_49.png
1_fig_50.png
1_fig_51.png
1_fig_52.png
1_fig_53.png
1_fig_54.png
1_fig_55.png
1_fig_56.png
1_fig_57.png
1_fig_58.png
1_fig_59.png
1_fig_60.png
1_fig_61.png
2_fig_1.png
2_fig_2.png
2_fig_3.png
2_fig_4.png
2_fig_5.png
2_fig_6.png
2_fig_7.png
2_fig_8.png
2_fig_9.png
2_fig_10.png
2_fig_11.png
2_fig_12.png
2_fig_13.png
2_fig_14.png
2_fig_15.png
2_fig_16.png
2_fig_17.png
2_fig_18.png
2_f

5_fig_15.png
5_fig_16.png
5_fig_17.png
5_fig_18.png
5_fig_19.png
5_fig_20.png
5_fig_21.png
5_fig_22.png
5_fig_23.png
5_fig_24.png
5_fig_25.png
5_fig_26.png
5_fig_27.png
5_fig_28.png
5_fig_29.png
5_fig_30.png
5_fig_31.png
5_fig_32.png
5_fig_33.png
5_fig_34.png
5_fig_35.png
5_fig_36.png
5_fig_37.png
5_fig_38.png
5_fig_39.png
5_fig_40.png
5_fig_41.png
5_fig_42.png
5_fig_43.png
5_fig_44.png
5_fig_45.png
5_fig_46.png
5_fig_47.png
5_fig_48.png
5_fig_49.png
5_fig_50.png
5_fig_51.png
5_fig_52.png
5_fig_53.png
5_fig_54.png
5_fig_55.png
5_fig_56.png
5_fig_57.png
5_fig_58.png
5_fig_59.png
5_fig_60.png
